In [3]:
import collections

import numpy as np
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

#### В качестве датасетов будут использованы 2 небольших словаря, содержащих английские предложения и их перевод на французский


In [ ]:
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()
    return data.split('\n')

In [4]:
e_data = load_data('small_vocab_en')
f_data = load_data('small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


Строки в английском словаре соответствуют своему в переводу во французском

In [5]:
for i in range(2):
    print(f'small_vocab_en Line {i + 1}:  {e_data[i]}')
    print(f'small_vocab_fr Line {i + 1}:  {f_data[i]}')

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


### Данные
Сложность обработки напрямую зависит от сложности данных

In [6]:
english_words_counter = collections.Counter([word for sentence in e_data for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in f_data for word in sentence.split()])

print(f'{len([word for sentence in e_data for word in sentence.split()])} English words.')
print(f'{len(english_words_counter)} unique English words.')
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print(f'{len([word for sentence in f_data for word in sentence.split()])} French words.')
print(f'{len(french_words_counter)} unique French words.')
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Препроцессинг
Для работы с данными все слова будут конвертированы в числа методом:
1. Токенизирования слов в соответствующие уникальные идентификаторы
2. Добавления отступов для уравнения длины предложений

### Токенизация
Ниже производится токенизация, призваивающая каждому слову свой уникальный идентификатор, используя keras.tokenize, и выводится пример результата

In [7]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print(f'Sequence {sample_i + 1} in x')
    print(f'  Input:  {sent}')
    print(f'  Output: {token_sent}')

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Создание отступов
С помощью pad_sequences производится создание отступов для уравнения длины каждого предложения

In [8]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print(f'Sequence {sample_i + 1} in x')
    print(f'  Input:  {np.array(token_sent)}')
    print(f'  Output: {pad_sent}')

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Пайплайны 
Построение пайплайнов препроцессинга

In [9]:
def preprocess(x, y):
	preprocess_x, x_tk = tokenize(x)
	preprocess_y, y_tk = tokenize(y)

	preprocess_x = pad(preprocess_x)
	preprocess_y = pad(preprocess_y)
	preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

	return preprocess_x, preprocess_y, x_tk, y_tk

preproc_e_data, preproc_f_data, english_tokenizer, french_tokenizer =\
	preprocess(e_data, f_data)
	
max_english_sequence_length = preproc_e_data.shape[1]
max_french_sequence_length = preproc_f_data.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Модели
Ниже происходит работа с 4 моделями
- Модель 1 простейшаая
- Модель 2 со встраиванием
- Модель 3 бинаправленная

### Получение слов по id
Модель будет превращать полученные слова в их id, функция `logits_to_text` будет возвращать слова по их идентификатору

In [10]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: Простейшая
![RNN](images/rnn.png)
Простейшая модель является отличной основой для анализа данных, поступающих один за другим. Ниже происходит перевод с английского на французский

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

tmp_x = pad(preproc_e_data, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_f_data.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_f_data, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 82us/step - loss: 3.5162 - acc: 0.4027 - val_loss: nan - val_acc: 0.4516
Epoch 2/10
110288/110288 [==============================] - 7s 64us/step - loss: 2.4823 - acc: 0.4655 - val_loss: nan - val_acc: 0.4838
Epoch 3/10
110288/110288 [==============================] - 7s 63us/step - loss: 2.2427 - acc: 0.5016 - val_loss: nan - val_acc: 0.5082
Epoch 4/10
110288/110288 [==============================] - 7s 64us/step - loss: 2.0188 - acc: 0.5230 - val_loss: nan - val_acc: 0.5428
Epoch 5/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.8418 - acc: 0.5542 - val_loss: nan - val_acc: 0.5685
Epoch 6/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.7258 - acc: 0.5731 - val_loss: nan - val_acc: 0.5811
Epoch 7/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.6478 - acc: 0.5871 - val_loss: nan - val_acc: 0.58

### Модель 2: С вложениями
![RNN](images/embedding.png)
Иным вриантом представления слов, помимо id, является векторное представление, т.е. вложения.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tmp_x = pad(preproc_e_data, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_f_data.shape[-2]))


embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embeded_model.fit(tmp_x, preproc_f_data, batch_size=1024, epochs=10, validation_split=0.2)


print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 68us/step - loss: 3.7877 - acc: 0.4018 - val_loss: nan - val_acc: 0.4093
Epoch 2/10
110288/110288 [==============================] - 7s 65us/step - loss: 2.7258 - acc: 0.4382 - val_loss: nan - val_acc: 0.5152
Epoch 3/10
110288/110288 [==============================] - 7s 65us/step - loss: 2.0359 - acc: 0.5453 - val_loss: nan - val_acc: 0.6068
Epoch 4/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.4586 - acc: 0.6558 - val_loss: nan - val_acc: 0.6967
Epoch 5/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.1346 - acc: 0.7308 - val_loss: nan - val_acc: 0.7561
Epoch 6/10
110288/110288 [==============================] - 7s 65us/step - loss: 0.9358 - acc: 0.7681 - val_loss: nan - val_acc: 0.7825
Epoch 7/10
110288/110288 [==============================] - 7s 65us/step - loss: 0.8057 - acc: 0.7917 - val_loss: nan - val_acc: 0.79

### Модель 3: Бинаправленная
![RNN](images/bidirectional.png)
Одним из важных ограничений предыдущих моделей является невозможность получения информации о будущих данных, только о прошлых. Данная модель решает эту проблему

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

tmp_x = pad(preproc_e_data, preproc_f_data.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_f_data.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    preproc_f_data.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)


bidi_model.fit(tmp_x, preproc_f_data, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 12s 110us/step - loss: 2.6981 - acc: 0.5009 - val_loss: 1.7832 - val_acc: 0.5699
Epoch 2/20
110288/110288 [==============================] - 12s 105us/step - loss: 1.6524 - acc: 0.5878 - val_loss: 1.4746 - val_acc: 0.6116
Epoch 3/20
110288/110288 [==============================] - 12s 105us/step - loss: 1.4406 - acc: 0.6161 - val_loss: 1.3488 - val_acc: 0.6278
Epoch 4/20
110288/110288 [==============================] - 12s 104us/step - loss: 1.3333 - acc: 0.6338 - val_loss: 1.2889 - val_acc: 0.6334
Epoch 5/20
110288/110288 [==============================] - 12s 104us/step - loss: 1.2650 - acc: 0.6462 - val_loss: 1.2543 - val_acc: 0.6357
Epoch 6/20
110288/110288 [==============================] - 12s 105us/step - loss: 1.2147 - acc: 0.6547 - val_loss: 1.2293 - val_acc: 0.6395
Epoch 7/20
110288/110288 [==============================] - 12s 105us/step - loss: 1.1708 - acc: 0.6627

### Модель 4: Объединенная
Включает в себя методы предыдущих для более точных результатов

In [15]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

print('Final Model Loaded')

Final Model Loaded


## Предсказание

In [17]:
def final_predictions(x, y, x_tk, y_tk):
    tmp_X = pad(preproc_e_data)
    model = model_final(tmp_X.shape,
                        preproc_f_data.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_f_data, batch_size = 1024, epochs = 17, validation_split = 0.2)
 
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_e_data, preproc_f_data, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/17
110288/110288 [==============================] - 41s 368us/step - loss: 1.9803 - acc: 0.5433 - val_loss: 1.2438 - val_acc: 0.6632
Epoch 2/17
110288/110288 [==============================] - 39s 354us/step - loss: 0.9968 - acc: 0.7186 - val_loss: 0.7820 - val_acc: 0.7716
Epoch 3/17
110288/110288 [==============================] - 39s 355us/step - loss: 0.6376 - acc: 0.8093 - val_loss: 0.4586 - val_acc: 0.8657
Epoch 4/17
110288/110288 [==============================] - 39s 355us/step - loss: 0.3872 - acc: 0.8854 - val_loss: 0.2809 - val_acc: 0.9183
Epoch 5/17
110288/110288 [==============================] - 39s 355us/step - loss: 0.2181 - acc: 0.9373 - val_loss: 0.1824 - val_acc: 0.9483
Epoch 6/17
110288/110288 [==============================] - 39s 354us/step - loss: 0.1565 - acc: 0.9553 - val_loss: 0.1671 - val_acc: 0.9513
Epoch 7/17
110288/110288 [==============================] - 39s 354us/step - loss: 0.1268 - acc: 0.9633